In [1]:
import os
import json
import soundfile as sf
import pyworld as pw
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
aiueo = ['a','i','u','e','o']

all_data = []
wav_dir = 'gliss_dataset'
out_dir = 'all_out'
note_step=0.25
frame_period = 10.0 # millisecond
out_dir = 'all_out'
def get_f0_index_tbl(f0,min_note,max_note,note_step=0.25):
    f0_note = 12*np.log2(f0/440)+49
    out_len = int((max_note-min_note)/note_step+1)
    note_lin =np.linspace(min_note, max_note, num=out_len)
    idx_out = np.zeros(out_len,dtype = int)
    for i in range(out_len):
        idx_out[i] = np.argmin(abs(f0_note-note_lin[i]))
    return idx_out

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for who in os.listdir(wav_dir):  
    rangePath = os.path.join(wav_dir,who,"info.json")
    with open(rangePath, 'r') as f:
        info = json.load(f)
        [min_note,max_note] = info['pRange']
    print(min_note,max_note)
    yy = np.array([0])
    for v in aiueo:
        path = os.path.join(wav_dir,who,v)
        print(path)
        x,fs = sf.read(path+'.wav')
        x = (x[:,0] + x[:,1])/2 
        x = x/max(x)*0.5
        f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)
        f0 = f0[1:-1]
        sp = sp[1:-1,:]
        ap = ap[1:-1]
        file_cnt = 2
        while os.path.isfile(path+str(file_cnt)+'.wav'):
            print(path+str(file_cnt)+'.wav')
            x2,_ = sf.read(path+str(file_cnt)+'.wav')
            x2 = (x2[:,0] + x2[:,1])/2 # stereo to mono
            f02, sp2,ap2 = pw.wav2world(x2, fs, frame_period =  frame_period)    # use default options

            f0 =np.concatenate((f0,f02[1:-1]), axis=0)
            sp =np.concatenate((sp,sp2), axis=0)
            ap = np.concatenate((ap,ap2), axis=0)
            file_cnt +=1
    #   remove f0 detect fail frame
        valid_idx = np.where(f0!=0)
        f0 =  f0[valid_idx]
        sp =  sp[valid_idx]
        ap =  ap[valid_idx]


        idx = get_f0_index_tbl(f0,min_note,max_note,note_step)  
        sp =  sp[idx,:]
        ap = ap[idx]

        lin_f0 = np.linspace(min_note, max_note,num=len(idx))
        f0 = 440*np.power(2,(lin_f0-49.0)/12.0)
    #     _y = pw.synthesize(f0, sp, ap[:len(idx)], fs, frame_period = 20)
        _y = pw.synthesize(f0, sp, ap, fs, frame_period = 20)

        yy = np.concatenate((yy,_y),axis = 0)

    outpath = os.path.join(out_dir,who+'.wav')
    sf.write(outpath, yy, fs)

26 39
gliss_dataset\Abun\a
gliss_dataset\Abun\a2.wav
gliss_dataset\Abun\i
gliss_dataset\Abun\u


KeyboardInterrupt: 